In [13]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline 

In [14]:
df = pd.read_csv('bought_insurance.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df[['age', 'affordibility']], df.bought_insurance, test_size=0.2, random_state=0)

In [16]:
X_train_scaled = X_train.copy()
X_train_scaled['age'] = X_train_scaled['age'] / 100 # I scale my age which is between 1 and 100

X_test_scaled = X_test.copy()
X_test_scaled['age'] = X_test_scaled['age'] / 100

In [17]:
X_test_scaled # Now I have my age scaled 

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [18]:
model = keras.Sequential([
    keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros') # My input_shape is age and affordibility,so its 2
]) # kernel_initializer its  weights which is one and bias_initializer its zero
model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy',
              metrics = ['accuracy'])

model.fit(X_train_scaled, y_train, epochs=5000)

Strumieniowane dane wyjściowe obcięte do 5000 ostatnich wierszy.
1/1 [==============================] - 0s 31ms/step - loss: 0.5450 - accuracy: 0.8636
Epoch 2502/5000
1/1 [==============================] - 0s 37ms/step - loss: 0.5450 - accuracy: 0.8636
Epoch 2503/5000
1/1 [==============================] - 0s 35ms/step - loss: 0.5449 - accuracy: 0.8636
Epoch 2504/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5449 - accuracy: 0.8636
Epoch 2505/5000
1/1 [==============================] - 0s 19ms/step - loss: 0.5448 - accuracy: 0.8636
Epoch 2506/5000
1/1 [==============================] - 0s 20ms/step - loss: 0.5448 - accuracy: 0.8636
Epoch 2507/5000
1/1 [==============================] - 0s 24ms/step - loss: 0.5447 - accuracy: 0.8636
Epoch 2508/5000
1/1 [==============================] - 0s 21ms/step - loss: 0.5447 - accuracy: 0.8636
Epoch 2509/5000
1/1 [==============================] - 0s 15ms/step - loss: 0.5446 - accuracy: 0.8636
Epoch 2510/5000
1/1 [============

In [19]:
model.evaluate(X_test_scaled, y_test)

1/1 [==============================] - 0s 257ms/step - loss: 0.4539 - accuracy: 1.0000


[0.4538957178592682, 1.0]

In [20]:
X_test_scaled

,age,affordibility
2,0.47,1
22,0.40,1
14,0.49,1
17,0.58,1
5,0.56,1
11,0.28,1


In [21]:
model.predict(X_test_scaled)

1/1 [==============================] - 0s 85ms/step


array([[0.6177279],
       [0.5396322],
       [0.6391427],
       [0.727986 ],
       [0.7094467],
       [0.4033524]], dtype=float32)

In [22]:
y_test

2     1
22    1
14    1
17    1
5     1
11    0
Name: bought_insurance, dtype: int64

In [23]:
coef, intercept = model.get_weights()
coef, intercept

(array([[4.5864797],
        [1.1000603]], dtype=float32),
 array([-2.7757905], dtype=float32))

In [24]:
def sigmoid(x): # It is my activation function
  import math
  return 1/(1+math.exp(-x))

In [25]:
def prediction_function(age, affordibility):
  weigthed_sum = coef[0]*age + coef[1]*affordability + intercept # Intercept is bias 
  return sigmoid(weigthed_sum)

In [26]:
prediction_function(0.47,1)

0.617727892902254

In [28]:
prediction_function(0.40,1)

0.5396321538833132

In [29]:
def log_loss(y_true, y_pred): # log loss is binary crossentropy 
  epsilon = 1e-15
  y_pred_new = [max(i,epsilon)for i in y_pred]
  y_pred_new = [min(i,1-epsilon)for i in y_pred_new]
  y_pred_new = np.array(y_pred_new)

  return -np.mean(y_true*np.log(y_pred_new)+(1-y_true)*np.log(1-y_pred_new))

In [30]:
def sigmoid_numpy(X):
  return 1/(1+np.exp(-X))

sigmoid_numpy(np.array([12,0,1])) # This function take array instead of 1 value, so It return sigmoid of entire array

array([0.99999386, 0.5       , 0.73105858])

In [37]:
def gradient_descent(age, affordibility, y_true, epochs, loss_thresold): # This function help finding weights 
  w1 = w2 = 1 # I save it as a random value
  bias = 0 # also this 
  learning_rate = 0.5 # here its try and error
  n = len(age) # Its length of samples 

  for i in range(epochs):
    weighted_sum = w1 * age + w2 * affordibility + bias # Its from pattern, so it give me weighted_sum on which i can apply sigmoid function
    y_pred = sigmoid_numpy(weighted_sum) # I have firs part of neuron weighted_sum and activation function as sigmoid_numpy

    loss = log_loss(y_true, y_pred) # Here it calculate loss 

    w1d = (1/n)*np.dot(np.transpose(age),(y_pred-y_true)) # Its pattern for my first derivative, I apply It on my age, my age is "xi" | np.dot just sum it up | (1/n) is average 
    w2d = (1/n)*np.dot(np.transpose(affordibility),(y_pred-y_true)) # Next derivative I apply on affordibility which is also "xi" | and (1/n) is average

    bias_d = np.mean(y_pred-y_true) # And here is bias derivative | np.mean return my average of y_pred-y_true
 
    w1 = w1 - learning_rate * w1d # It is a pattern for w1
    w2 = w2 - learning_rate * w2d # And w2

    bias = bias - learning_rate * bias_d # also for my bias 

    print(f'Epoch: {1}, w1:{w1}, w2:{w2}, bias:{bias}, loss:{loss}') # Here every epoch it print my parameters 

    if loss<= loss_thresold: # I will stop my loss in this same point as my tensorflow model 
      break # I will pass my loss value to my function below 

  return w1, w2, bias

In [38]:
gradient_descent(X_train_scaled['age'], X_train_scaled['affordibility'], y_train, 1000,0.4606) # Here from X_train_scaled it is an entire row (age and affordibility)

Epoch: 1, w1:0.9644915668168719, w2:0.9224250019701856, bias:-0.1521744437835525, loss:0.7960362503355486
Epoch: 1, w1:0.9366743363654499, w2:0.8570969777186894, bias:-0.2838815971644586, loss:0.7394293670708099
Epoch: 1, w1:0.9160190930945575, w2:0.8035200242324965, bias:-0.39666489545168215, loss:0.6980698848391689
Epoch: 1, w1:0.9017701861063158, w2:0.7606695089288332, bias:-0.4926120466423785, loss:0.6686193819406211
Epoch: 1, w1:0.8930731543176171, w2:0.7272344982483193, bias:-0.5740228037875161, loss:0.6479960845015644
Epoch: 1, w1:0.8890759364765383, w2:0.7018265973050505, bias:-0.6431543649176596, loss:0.6336578618954344
Epoch: 1, w1:0.8889918350364353, w2:0.6831195783238424, bias:-0.7020686534913516, loss:0.6236667598398874
Epoch: 1, w1:0.8921292648124783, w2:0.6699231996615576, bias:-0.7525646139800298, loss:0.6166236529551378
Epoch: 1, w1:0.8978991061148434, w2:0.6612102292494013, bias:-0.796166156177148, loss:0.6115548071590622
Epoch: 1, w1:0.9058095436860382, w2:0.65611589

(4.562607017100527, 1.1793032507081156, -2.844701939090328)

In [39]:
coef, intercept 

(array([[4.5864797],
        [1.1000603]], dtype=float32),
 array([-2.7757905], dtype=float32))

In [ ]:
w1 = 4.5864797, w2 = 1.1000603, bias = -2.7757905